<a href="https://colab.research.google.com/github/xutian1113/pytorch_practice/blob/main/WikiText_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install tqdm

## WikiText Description
The WikiText dataset is a collection of high-quality Wikipedia articles designed for language modeling tasks. It is commonly used to train and evaluate text generation models such as GPT, LLaMA, and Transformer-based models.

- Developed by: Salesforce Research
- Purpose: Train and benchmark language models (next-word prediction, text generation, etc.).
- Dataset Variants:
  - WikiText-2 (smaller, ideal for quick training)
  - WikiText-103 (larger, used for pretraining large models)
  - WikiText-2-raw and WikiText-103-raw (uncleaned versions)

### Key Features of WikiText
| **Feature**      | **Description** |
|---------------|----------------------|
|Task Type|	Language Modeling (Next-Word Prediction)|
|Input Type|	Continuous Wikipedia Text|
|Vocabulary|	~267K unique words (WikiText-103)|
|Dataset Size|	WikiText-2: 2M tokens, WikiText-103: 103M tokens|
|Text Type|	Cleaned Wikipedia articles (excludes lists, tables, headers)|
|Evaluation Metrics|	Perplexity (PPL), BPC (bits-per-character)|
|Commonly Used With|	GPT, Transformer models, LSTMs|


### WikiText Variants and Size

| Dataset |Tokens|Train Samples|Validation Samples|Test Samples|
|---------|------|-------------|------------------|------------|
|WikiText-2	|2M	|36,718	|3,760	|4,358|
|WikiText-103|	103M|	28,475|	60|	60|
|WikiText-2-raw|	2M|	36,718|	3,760	|4,358|
|WikiText-103-raw	|103M|	28,475|	60|	60|

- WikiText-2: Smaller version for experiments and fine-tuning.
- WikiText-103: Large dataset for training large-scale transformers.
- Raw versions: Include additional formatting (unprocessed text)


### Example Sentences from WikiText

| **Article**      | **Excerpt** |
|---------------|----------------------|
|	Artificial Intelligence |	"Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by humans and animals."|
|Physics	|"Physics is the natural science that studies matter, its fundamental constituents, its motion and behavior through space and time, and the related entities of energy and force."|
|History of Computing|	"The history of computing is longer than the history of computing hardware and modern computing technology and includes the history of methods intended for pen and paper or for chalk and slate."|




In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
def accuracy(preds, labels):
  _, predicted = torch.max(preds, 1)
  correct = (predicted == labels).sum().item()
  return correct / len(labels)

def f1_score(preds, labels):
  _, predicted = torch.max(preds, 1)
  tp = ((predicted == 1) & (labels == 1)).sum().item()
  fp = ((predicted == 1) & (labels == 0)).sum().item()
  fn = ((predicted == 0) & (labels == 1)).sum().item()
  precision = tp / (tp + fp) if tp + fp > 0 else 0
  recall = tp / (tp + fn) if tp + fn > 0 else 0
  f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
  return f1

def precision(preds, labels):
  _, predicted = torch.max(preds, 1)
  tp = ((predicted == 1) & (labels == 1)).sum().item()
  fp = ((predicted == 1) & (labels == 0)).sum().item()
  precision = tp / (tp + fp) if tp + fp > 0 else 0
  return precision


def auc_roc(preds, labels):
  # Convert predictions and labels to NumPy arrays
  _, predicted = torch.max(preds, 1)
  preds_np = predicted.detach().cpu().numpy()
  labels_np = labels.detach().cpu().numpy()

  # Calculate AUC-ROC using scikit-learn's roc_auc_score
  auc_score = roc_auc_score(labels_np, preds_np)

  return auc_score


def auc_pr(preds, labels):
  """
  Calculates the AUPRC score for binary classification.

  Args:
    preds: Predicted probabilities (tensor) for the positive class.
    labels: True labels (tensor) (0 or 1).

  Returns:
    AUPRC score (float).
  """
  # Convert predictions and labels to NumPy arrays
  _, predicted = torch.max(preds, 1)
  preds_np = predicted.detach().cpu().numpy()
  labels_np = labels.detach().cpu().numpy()

  # Calculate AUPRC using scikit-learn's average_precision_score
  auprc_score = average_precision_score(labels_np, preds_np)

  return auprc_score

def confusion_matrix_func(preds, labels, num_classes=2):
  """
  Calculates the confusion matrix.

  Args:
    preds: Predicted labels (tensor).
    labels: True labels (tensor).
    num_classes: Number of classes. Defaults to 2 for binary classification.

  Returns:
    Confusion matrix (NumPy array).
  """
  # Get predicted class indices
  _, predicted = torch.max(preds, 1)

  # Convert predictions and labels to NumPy arrays
  predicted_np = predicted.detach().cpu().numpy()
  labels_np = labels.detach().cpu().numpy()

  # Calculate confusion matrix using scikit-learn's confusion_matrix
  cm = confusion_matrix(labels_np, predicted_np, labels=range(num_classes))

  return cm

def recall(preds, labels):
  """
  Calculates the recall score for binary classification.

  Args:
    preds: Predicted labels (tensor).
    labels: True labels (tensor).

  Returns:
    Recall score (float).
  """
  # Get predicted class indices
  _, predicted = torch.max(preds, 1)

  # Calculate true positives (TP), false negatives (FN)
  tp = ((predicted == 1) & (labels == 1)).sum().item()
  fn = ((predicted == 0) & (labels == 1)).sum().item()

  # Calculate recall
  recall_score = tp / (tp + fn) if tp + fn > 0 else 0  # Avoid division by zero

  return recall_score

def MCC(preds, labels):
  """
  Calculates the Matthews correlation coefficient (MCC) for binary classification.

  Args:
    preds: Predicted labels (tensor).
    labels: True labels (tensor).

  Returns:
    MCC score (float).
    """
  # Get predicted class indices
  _, predicted = torch.max(preds, 1)

  # Convert predictions and labels to NumPy arrays
  predicted_np = predicted.detach().cpu().numpy()
  labels_np = labels.detach().cpu().numpy()

  # Calculate MCC using scikit-learn's matthews_corrcoef
  mcc_score = matthews_corrcoef(predicted_np, labels_np)
  return mcc_score


In [5]:
# Load WikiText-2 dataset (other options: "wikitext-103", "wikitext-2-raw")
dataset = load_dataset("wikitext", "wikitext-2-v1")



```
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})
```

In [10]:
from transformers import GPT2Tokenizer

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Assign a padding token (GPT-2 does not have one by default)
tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove original text column
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [11]:
from transformers import GPT2LMHeadModel

# Load GPT-2 model for text generation
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Move model to GPU if available
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [12]:
from torch.utils.data import DataLoader

batch_size = 8

train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=batch_size)


In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Training Loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}"):
        batch = {key: value.to(device) for key, value in batch.items()}
        optimizer.zero_grad()

        outputs = model(**batch, labels=batch["input_ids"])
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_loss:.4f}")


Training Epoch 1: 100%|██████████| 4590/4590 [05:10<00:00, 14.81it/s]


Epoch 1, Training Loss: 1.2200


Training Epoch 2:   0%|          | 0/4590 [00:00<?, ?it/s]

In [ ]:
import math

model.eval()
total_loss = 0

with torch.no_grad():
    for batch in tqdm(val_dataloader, desc=f"Validating"):
        batch = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**batch, labels=batch["input_ids"])
        total_loss += outputs.loss.item()

avg_loss = total_loss / len(val_dataloader)
perplexity = math.exp(avg_loss)

print(f"Validation Perplexity: {perplexity:.2f}")


In [ ]:
# Testing
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate 50 tokens
output = model.generate(input_ids, max_length=50, num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


In [ ]:
# 1. load the GLUE MRPC dataset
dataset = load_dataset('glue', 'cola')

# 2. load the tokenizer for bert-base-cased
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# converting raw text into numerical representations that can be processed by BERT model.

# 3. define a tokenization function for sentence pair

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True, padding="max_length", max_length=128)

'''
input_ids: Tokenized numerical representation of both sentences.
token_type_ids: Differentiates sentence1 (0) from sentence2 (1).
attention_mask: Indicates which tokens are real (1) and padding (0).
'''
# 4. Tokenize the dataset (batched processing)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 5. Remove unnecessary columns and set the format to PyTorch tensors
# Remove columns that are not used by the model (like the original sentences and index)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets.set_format("torch")

# 6. Create DataLoaders for training and validation
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8)
val_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=8)


# 7. Set up the device, model, optimizer, and scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
model.to(device)

# optimizer = AdamW(model.parameters(), lr=2e-5)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
epochs = 20
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
best_val_loss = float('inf')
best_model_state = None
best_epoch = 0
epochs_no_improve = 0
patience = 3

model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    total_loss = 0
    all_preds = []
    all_labels = []
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}"):
        # Move batch to the device
        batch = {key: value.to(device) for key, value in batch.items()}
        batch['labels'] = batch['label']
        del batch['label']
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        all_preds.append(outputs.logits.detach())
        all_labels.append(batch['labels'].detach())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    train_acc = accuracy(all_preds, all_labels)
    train_auc = auc_roc(all_preds, all_labels)
    train_rec = recall(all_preds, all_labels)
    train_avg_loss = total_loss / len(train_dataloader)
    train_mcc = MCC(all_preds, all_labels)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {train_avg_loss:.4f}, Accuracy: {train_acc:.4f}, AUC-ROC: {train_auc:.4f}, Recall: {train_rec:.4f}, MCC: {train_mcc:.4f}")

    # 9. Validation loop
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    val_preds = []
    val_labels = []
    val_loss = 0
    for batch in val_dataloader:
        batch = {key: value.to(device) for key, value in batch.items()}
        batch['labels'] = batch['label']
        del batch['label']
        with torch.no_grad():
            outputs = model(**batch)
            val_loss += outputs.loss.item()
        logits = outputs.logits
        val_preds.append(logits.detach())
        val_labels.append(batch['labels'].detach())

    val_preds = torch.cat(val_preds)
    val_labels = torch.cat(val_labels)
    val_acc = accuracy(val_preds, val_labels)
    val_auc = auc_roc(val_preds, val_labels)
    val_rec = recall(val_preds, val_labels)
    val_avg_loss = val_loss / len(val_dataloader)
    val_mcc = MCC(val_preds, val_labels)
    if val_avg_loss < best_val_loss:
        best_val_loss = val_avg_loss
        best_model_state = model.state_dict()
        best_epoch = epoch
        epochs_no_improve = 0
    else:
      epochs_no_improve += 1
      # if epochs_no_improve >= patience:
      #     print(f"Early stopping at epoch {epoch+1} due to no improvement in validation loss for {patience} epochs.")
      #     break
    print(f"Validation - Loss: {val_avg_loss:.4f},  Accuracy: {val_acc:.4f}, AUC-ROC: {val_auc:.4f}, Recall: {val_rec:.4f}, MCC: {val_mcc:.4f}")


    model.train()  # Switch back to training mode for the next epoch


Epoch 1/20


Training Epoch 1: 100%|██████████| 1069/1069 [00:51<00:00, 20.67it/s]


Epoch 1/20 - Loss: 0.4989, Accuracy: 0.7659, AUC-ROC: 0.6699, Recall: 0.9047, MCC: 0.3908
Validation - Loss: 0.4441,  Accuracy: 0.8006, AUC-ROC: 0.7028, Recall: 0.9584, MCC: 0.5026
Epoch 2/20


Training Epoch 2: 100%|██████████| 1069/1069 [00:51<00:00, 20.88it/s]


Epoch 2/20 - Loss: 0.2825, Accuracy: 0.8862, AUC-ROC: 0.8553, Recall: 0.9309, MCC: 0.7229
Validation - Loss: 0.3951,  Accuracy: 0.8265, AUC-ROC: 0.7894, Recall: 0.8863, MCC: 0.5878
Epoch 3/20


...

Epoch 9/20 - Loss: 0.0289, Accuracy: 0.9891, AUC-ROC: 0.9869, Recall: 0.9924, MCC: 0.9739
Validation - Loss: 0.9854,  Accuracy: 0.8063, AUC-ROC: 0.7379, Recall: 0.9168, MCC: 0.5222
Epoch 10/20


Training Epoch 10: 100%|██████████| 1069/1069 [00:51<00:00, 20.87it/s]


Epoch 10/20 - Loss: 0.0204, Accuracy: 0.9917, AUC-ROC: 0.9897, Recall: 0.9945, MCC: 0.9801
Validation - Loss: 1.2007,  Accuracy: 0.8111, AUC-ROC: 0.7396, Recall: 0.9265, MCC: 0.5332
Epoch 11/20


Training Epoch 11:  29%|██▊       | 306/1069 [00:14<00:36, 20.86it/s]


KeyboardInterrupt: 

In [ ]:
test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8)
model.eval()
test_preds = []
test_labels = []

for batch in test_dataloader:
    batch = {key: value.to(device) for key, value in batch.items()}
    batch['labels'] = batch['label']
    del batch['label']
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    test_preds.append(logits.detach())
    test_labels.append(batch['labels'].detach())

test_preds = torch.cat(test_preds)
test_labels = torch.cat(test_labels)

test_acc = accuracy(test_preds, test_labels)
test_auc = auc_roc(test_preds, test_labels)
test_rec = recall(test_preds, test_labels)
print(f"Test - Accuracy: {test_acc:.4f}, AUC-ROC: {test_auc:.4f}, Recall: {test_rec:.4f}")